In [78]:
from sklearn.preprocessing import normalize
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import hopsworks
import os

2025-01-06 03:58:25.154284: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 03:58:25.165855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736132305.177388   25481 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736132305.181103   25481 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 03:58:25.193339: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
if os.getenv('HOPSWORKS_API_KEY') is not None:
    HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')
else:
    with open('../secrets/hopsworks_api_key.txt', 'r') as file:
        HOPSWORKS_API_KEY = file.readline().strip()

In [3]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store() 

2025-01-06 03:58:28,560 INFO: Initializing external client
2025-01-06 03:58:28,561 INFO: Base URL: https://c.app.hopsworks.ai:443

Multiple projects found. 

	 (1) id2223_final_project
	 (2) id2223_lab1_group9
	 (3) lab1_mohamed_Emile



Enter number corresponding to the project to use:  1


2025-01-06 03:58:33,827 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1208515


In [4]:
user_embeddings_fg = fs.get_feature_group(
    name='spotify_user_embeddings',
    version=2,
)

user_embeddings_df = user_embeddings_fg.read()
print(f"A total of {len(user_embeddings_df)} user embeddings are available.")
user_embeddings_df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.51s) 
A total of 102 user embeddings are available.


,user_id,genre_embedding,artist_embedding,playlist_embedding,release_year_embedding
0,31imc4msmvetbl26gly5n55jbkka,"[0.1219930648803711, 0.1214483454823494, -0.27...","[0.28332215547561646, -0.15212738513946533, -0...","[0.1219930648803711, 0.1214483454823494, -0.27...",[2018.0]
1,31tgsl3dejcqihle3pv7o6eeng2a,"[-0.3863714933395386, -0.5621631145477295, 0.1...","[0.3942939341068268, -0.33002883195877075, 0.3...","[-0.7064403295516968, -0.9494979381561279, 0.1...",[2019.9066666666668]
2,31fg5ma4zjh37mcqzto3xt2sxc3a,"[-0.31446775794029236, -0.2531762421131134, 0....","[-0.29996415972709656, 0.5447441339492798, 0.0...","[-0.31446775794029236, -0.2531762421131134, 0....",[2019.0]
3,31h7ml3xiavflj5n7d4av5u5xaie,"[-0.3711507022380829, -0.19814574718475342, -0...","[-0.09817744046449661, 0.1808864027261734, -0....","[-0.5167202949523926, -0.22468358278274536, -0...",[2019.8850574712644]
4,31frxab22c2ez34gnfggtqqsnope,"[0.14112409949302673, 0.12898339331150055, -0....","[-0.010701656341552734, 0.32390376925468445, -...","[0.26713827252388, 0.21868111193180084, -0.577...",[2017.2272727272727]


In [5]:
user_embeddings_df['full_embedding'] = user_embeddings_df.apply(
    lambda row: np.concatenate(
        [row['genre_embedding'], row['artist_embedding'], row['playlist_embedding'], row['release_year_embedding']]
    ),
    axis=1
)
normalized_embeddings = normalize(np.array(user_embeddings_df['full_embedding'].tolist()))
user_embeddings_df['normalized_embedding'] = normalized_embeddings.tolist()
user_embeddings_df.head()

,user_id,genre_embedding,artist_embedding,playlist_embedding,release_year_embedding,full_embedding,normalized_embedding
0,31imc4msmvetbl26gly5n55jbkka,"[0.1219930648803711, 0.1214483454823494, -0.27...","[0.28332215547561646, -0.15212738513946533, -0...","[0.1219930648803711, 0.1214483454823494, -0.27...",[2018.0],"[0.1219930648803711, 0.1214483454823494, -0.27...","[6.0451476847805844e-05, 6.018155091299676e-05..."
1,31tgsl3dejcqihle3pv7o6eeng2a,"[-0.3863714933395386, -0.5621631145477295, 0.1...","[0.3942939341068268, -0.33002883195877075, 0.3...","[-0.7064403295516968, -0.9494979381561279, 0.1...",[2019.9066666666668],"[-0.3863714933395386, -0.5621631145477295, 0.1...","[-0.00019127666431846473, -0.00027830387905733..."
2,31fg5ma4zjh37mcqzto3xt2sxc3a,"[-0.31446775794029236, -0.2531762421131134, 0....","[-0.29996415972709656, 0.5447441339492798, 0.0...","[-0.31446775794029236, -0.2531762421131134, 0....",[2019.0],"[-0.31446775794029236, -0.2531762421131134, 0....","[-0.0001557508143566418, -0.000125394113988495..."
3,31h7ml3xiavflj5n7d4av5u5xaie,"[-0.3711507022380829, -0.19814574718475342, -0...","[-0.09817744046449661, 0.1808864027261734, -0....","[-0.5167202949523926, -0.22468358278274536, -0...",[2019.8850574712644],"[-0.3711507022380829, -0.19814574718475342, -0...","[-0.00018374682362942734, -9.809667997751179e-..."
4,31frxab22c2ez34gnfggtqqsnope,"[0.14112409949302673, 0.12898339331150055, -0....","[-0.010701656341552734, 0.32390376925468445, -...","[0.26713827252388, 0.21868111193180084, -0.577...",[2017.2272727272727],"[0.14112409949302673, 0.12898339331150055, -0....","[6.995799678794909e-05, 6.393960951673316e-05,..."


In [6]:
def build_user_tower(input_dim, embedding_dim=128):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    user_embedding = layers.Dense(embedding_dim, activation=None)(x)  # Final user embedding
    return Model(inputs, user_embedding, name="UserTower")

def build_candidate_tower(input_dim, embedding_dim=128):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    candidate_embedding = layers.Dense(embedding_dim, activation=None)(x)  # Final candidate embedding
    return Model(inputs, candidate_embedding, name="CandidateTower")

def build_feature_cross_model(user_input_dim, candidate_input_dim, embedding_dim=128):
    # User tower
    user_tower = build_user_tower(user_input_dim, embedding_dim)
    user_embedding = user_tower.output

    # Candidate tower
    candidate_tower = build_candidate_tower(candidate_input_dim, embedding_dim)
    candidate_embedding = candidate_tower.output

    # Feature cross layer
    cross_layer = layers.Dot(axes=1)([user_embedding, candidate_embedding])
    cross_output = layers.Reshape((1,))(cross_layer)

    # Concatenate cross output with user embedding
    combined = layers.Concatenate()([user_embedding, cross_output])

    # Deep layers
    x = layers.Dense(256, activation='relu')(combined)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(8, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(4, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    # Final model
    model = Model(inputs=[user_tower.input, candidate_tower.input], outputs=x)
    return model

In [7]:
# Example usage
user_input_dim = len(normalized_embeddings[0])       # Example input dimension for user features
candidate_input_dim = len(normalized_embeddings[0])  # Example input dimension for candidate features
embedding_dim = 128

model = build_feature_cross_model(user_input_dim, candidate_input_dim, embedding_dim)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

2025-01-06 03:59:22.430136: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
def generate_pairs(embeddings, similarity_threshold=0.8, negative_ratio=1):
    pairs = []
    labels = []

    # Compute cosine similarity for all pairs
    similarity_matrix = cosine_similarity(embeddings)  # This is a valid pairwise similarity matrix

    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            if similarity_matrix[i, j] > similarity_threshold:
                # Positive pair
                pairs.append((embeddings[i], embeddings[j]))
                labels.append(1)

                # # Generate negative pairs
                # for _ in range(negative_ratio):
                #     negative_index = np.random.choice(len(embeddings))
                #     while negative_index == i or negative_index == j:
                #         negative_index = np.random.choice(len(embeddings))

                #     pairs.append((embeddings[i], embeddings[negative_index]))
                #     labels.append(0)

                # Generate semi-hard negatives
                for _ in range(negative_ratio):
                    semi_hard_found = False
                    for k in range(len(embeddings)):
                        if k != i and k != j:  # Avoid self-pairs or positive pairs
                            similarity = similarity_matrix[i, k]
                            if similarity_threshold/2 < similarity < similarity_threshold:
                                # Semi-hard negative found
                                pairs.append((embeddings[i], embeddings[k]))
                                labels.append(0)
                                semi_hard_found = True
                                break  # Stop after finding one semi-hard negative
                    
                    if not semi_hard_found:
                        # Fall back to a random negative if no semi-hard is found
                        negative_index = np.random.choice(len(embeddings))
                        while negative_index == i or negative_index == j:
                            negative_index = np.random.choice(len(embeddings))
                        pairs.append((embeddings[i], embeddings[negative_index]))
                        labels.append(0)
            
    
    return np.array(pairs), np.array(labels)

In [9]:
# Generate training data
pairs, labels = generate_pairs(normalized_embeddings)
user_1 = np.array([pair[0] for pair in pairs])
user_2 = np.array([pair[1] for pair in pairs])

history = model.fit(
    [user_1, user_2],  # Input: pairs of user embeddings
    labels,            # Output: similarity labels
    batch_size=32,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5046 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4801 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4904 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5008 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5008 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4879 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4979 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5044 - loss: 0.6931 - val_accuracy: 0

In [10]:
mr = project.get_model_registry()
model.save("two_tower_model.keras", save_format="tf")

# Create a new model version
model_dir = "two_tower_model.keras"
model_name = "two_tower_recommender"

model_registry = mr.python.create_model(
    name=model_name,
    metrics={"accuracy": history.history["accuracy"][-1]},  # Log the final accuracy
    description="Two-Tower Recommender Model for User Similarity",
)

model_registry.save(model_dir)
print(f"Model '{model_name}' uploaded to Hopsworks!")

2025-01-06 04:00:15,419 WARNING: The `save_format` argument is deprecated in Keras 3. We recommend removing this argument as it can be inferred from the file path. Received: save_format=tf


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/9326635 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1208515/models/two_tower_recommender/8
Model 'two_tower_recommender' uploaded to Hopsworks!
